# 0. Introduction

In this notebook we explore the different tables in the "Cruncbase 2013 snapshot"
The main questions we want to answer are: What data is available? What data is useful for modelling?
The output of this notebook is a dataset containing the data for further analysis

In [1]:
# Importing the libraries
import pandas as pd

In [16]:
# read in the different tables
acquisition_df = pd.read_csv(r"C:\Users\32474\Documents\Ma TEW\masterthesis\data\acquisitions.csv")
degrees_df = pd.read_csv(r"C:\Users\32474\Documents\Ma TEW\masterthesis\data\degrees.csv")
investments_df = pd.read_csv(r"C:\Users\32474\Documents\Ma TEW\masterthesis\data\investments.csv")
offices_df = pd.read_csv(r"C:\Users\32474\Documents\Ma TEW\masterthesis\data\offices.csv")
people_df = pd.read_csv(r"C:\Users\32474\Documents\Ma TEW\masterthesis\data\people.csv")
relationships_df = pd.read_csv(r"C:\Users\32474\Documents\Ma TEW\masterthesis\data\relationships.csv")
objects_df = pd.read_excel(r"C:\Users\32474\Documents\Ma TEW\masterthesis\data\object_reworked.xlsx")

We will look at each table that is available.
From this we can decide which data is useful, which data we need to fill and how we want to merge the data

# 1. Acquisitions

In [18]:
acquisition_df.shape

(9562, 12)

In [19]:
acquisition_df.head()

,id,acquisition_id,acquiring_object_id,acquired_object_id,term_code,price_amount,price_currency_code,acquired_at,source_url,source_description,created_at,updated_at
0,1,1,c:11,c:10,NaN,20000000.0,USD,2007-05-30,http://venturebeat.com/2007/05/30/fox-interact...,Fox Interactive confirms purchase of Photobuck...,2007-05-31 22:19:54,2008-05-21 19:23:44
1,2,7,c:59,c:72,cash,60000000.0,USD,2007-07-01,http://www.techcrunch.com/2007/07/02/deal-is-c...,Deal is Confirmed: Google Acquired GrandCentral,2007-07-03 08:14:50,2011-05-06 21:51:05
2,3,8,c:24,c:132,cash,280000000.0,USD,2007-05-01,http://www.techcrunch.com/2007/05/30/cbs-acqui...,CBS Acquires Europeâs Last.fm for $280 million,2007-07-12 04:19:24,2008-05-19 04:48:50
3,4,9,c:59,c:155,cash,100000000.0,USD,2007-06-01,http://techcrunch.com/2007/05/23/100-million-p...,$100 Million Payday For Feedburner  This Deal...,2007-07-13 09:52:59,2012-06-05 03:22:17
4,5,10,c:212,c:215,cash,25000000.0,USD,2007-07-01,http://blog.seattlepi.nwsource.com/venture/arc...,seatlepi.com,2007-07-20 05:29:07,2008-02-25 00:23:47


In [20]:
acquisition_df.isnull().sum()

id                        0
acquisition_id            0
acquiring_object_id       0
acquired_object_id        1
term_code              7656
price_amount              0
price_currency_code       4
acquired_at              29
source_url              996
source_description      972
created_at                0
updated_at                0
dtype: int64

In [21]:
len(acquisition_df[acquisition_df.price_amount != 0])

2599

**Conclusion**
We only have about 2600 M&A deals that contain a deal price
these deals are the starting point from our analysis. We can enrich this data by merging the acquirers and acquired firms and their relevant information.
We start by creating a deals dataframe that will be the basis. Next we create the acquirer and acquired dataframes, these can than be merged to it

In [22]:
# Create a dataframe that will be the basis of the analysis
df = acquisition_df.loc[acquisition_df.price_amount != 0]
df = df.drop(columns=["id", "acquisition_id", "term_code", "created_at", "updated_at", "source_url", "source_description"])
df = df.dropna()

In [23]:
df.shape

(2595, 5)

# 2.  Objects

Get the relevant acquirer and acquired companies and link them to the deals with a price higher than 0

In [24]:
# Select relevant columns
columns = ["id", "entity_type", "name", "category_code", "founded_at", "country_code", "region", "investment_rounds", "invested_companies",
                    "funding_rounds", "funding_total_usd", "milestones", "relationships"]
objects_df = objects_df.loc[:, columns]

In [25]:
# Get the acquiring firms
acquirers_df = pd.merge(objects_df, df, left_on="id", right_on="acquiring_object_id")
acquirers_df = acquirers_df.loc[:, columns]

In [26]:
# Get the acquired firms
acquired_df = pd.merge(objects_df, df, left_on="id", right_on="acquired_object_id")
acquired_df = acquired_df.loc[:, columns]

## 2.1. Acquirer

In [27]:
acquirers_df.shape

(2583, 13)

In [29]:
acquirers_df.isnull().sum()

id                      0
entity_type             0
name                    0
category_code         216
founded_at            689
country_code          172
region                  0
investment_rounds       0
invested_companies      0
funding_rounds          0
funding_total_usd       0
milestones              0
relationships           0
dtype: int64

In [35]:
# Get the year the company was founded in
acquirers_df["founded_year"] = acquirers_df["founded_at"].dt.year
acquirers_df = acquirers_df.drop("founded_at", axis = 1)

In [36]:
acquirers_df.founded_year.median()

1994.0

In [16]:
# fill the empty years with the median year (1994)
acquirers_df["category_code"].fillna("other", inplace=True)
acquirers_df["founded_year"].fillna(acquirers_df["founded_year"].median(), inplace=True)

##  2.2. Acquired

In [17]:
acquired_df.shape

(2592, 13)

In [30]:
acquired_df.isnull().sum()

id                       0
entity_type              0
name                     0
category_code          522
founded_at            1082
country_code           557
region                   0
investment_rounds        0
invested_companies       0
funding_rounds           0
funding_total_usd        0
milestones               0
relationships            0
dtype: int64

In [32]:
# Get the year the company was founded in
acquired_df["founded_year"] = acquired_df["founded_at"].dt.year
acquired_df = acquired_df.drop("founded_at", axis = 1)

In [33]:
acquired_df.founded_year.median()

2000.0

In [19]:
# fill the empty years with the median year (2000)
acquired_df["category_code"].fillna("other", inplace=True)
acquired_df["founded_year"].fillna(acquired_df["founded_year"].median(), inplace=True)

# 3. Investments

More than 700 companies have one or more VC investors affiliated with them.
We created a list with the number of investors that can be added to the acquired dataframe

In [20]:
VC_backed = pd.merge(investments_df, acquired_df, left_on="funded_object_id", right_on="id")

In [21]:
# Create a list with the number of investors in the companies
number_investors = VC_backed.groupby("funded_object_id")["investor_object_id"].nunique()

In [22]:
# Add this list to the aqcuired dataframe
acquired_df = pd.merge(acquired_df, number_investors, how="left", left_on="id", right_on="funded_object_id")
acquired_df.rename(columns={"investor_object_id": "number_of_investors"}, inplace=True)
acquired_df["number_of_investors"].fillna(0, inplace=True)

# 4. Offices

We get the number of offices for each acquirer and acquired company

## 4.1 Acquirers offices

In [23]:
offices_df.head()

,id,object_id,office_id,description,region,address1,address2,city,zip_code,state_code,country_code,latitude,longitude,created_at,updated_at
0,1,c:1,1,NaN,Seattle,710 - 2nd Avenue,Suite 1100,Seattle,98104,WA,USA,47.603122,-122.333253,NaN,NaN
1,2,c:3,3,Headquarters,SF Bay,4900 Hopyard Rd,Suite 310,Pleasanton,94588,CA,USA,37.692934,-121.904945,NaN,NaN
2,3,c:4,4,NaN,SF Bay,135 Mississippi St,NaN,San Francisco,94107,CA,USA,37.764726,-122.394523,NaN,NaN
3,4,c:5,5,Headquarters,SF Bay,1601 Willow Road,NaN,Menlo Park,94025,CA,USA,37.416050,-122.151801,NaN,NaN
4,5,c:7,7,NaN,SF Bay,Suite 200,654 High Street,Palo Alto,94301,CA,ISR,0.000000,0.000000,NaN,NaN


In [24]:
acquirers_offices = pd.merge(offices_df, acquirers_df, left_on="object_id", right_on="id")

In [25]:
acquirers_offices.drop_duplicates(inplace=True)

In [26]:
acquirers_number_offices = acquirers_offices.groupby("object_id").count()["office_id"].to_frame("number_of_offices")

In [27]:
acquirers_df = pd.merge(acquirers_df, acquirers_number_offices, how="left", left_on="id", right_on="object_id")

## 4.2 Acquired offices

In [28]:
acquired_offices = pd.merge(offices_df, acquired_df, left_on="object_id", right_on="id")

In [29]:
acquired_offices.drop_duplicates(inplace=True)

In [30]:
acquired_number_offices = acquired_offices.groupby("object_id").count()["office_id"].to_frame("number_of_offices")

In [31]:
acquired_df = pd.merge(acquired_df, acquired_number_offices, how="left", left_on="id", right_on="object_id")

# 5. Relationships, founders and degree

We have 1737 companies with at least one relationship (the number of relationships is already in the dataset)
We also are able to retrieve 1361 founders, of these founders we can get the degree

In [32]:
relations = pd.merge(relationships_df, acquired_df, left_on="relationship_object_id", right_on="id")

In [33]:
relations["title"].str.contains("founder|Founder").sum()

1361

In [34]:
founders = relations[relations["title"].str.contains("founder|Founder") == True][["person_object_id", 'relationship_object_id', "title"]]

In [35]:
founders.shape

(1361, 3)

In [36]:
founders_degree = pd.merge(founders, degrees_df, left_on="person_object_id", right_on="object_id")[["person_object_id", 'relationship_object_id', "title",
                                                                                                    "degree_type", "institution"]]

In [37]:
# mapping used to retrieve the degree
# the degree columns has many different values with similar meaning, these will be grouped together
def get_degree(degrees):
    # PhD
    if any("PhD" in x for x in degrees):
        return "PhD"
    if any("Ph.D." in x for x in degrees):
        return "PhD"

    # MBA
    if any("MBA" in x for x in degrees):
        return  "MBA"

    # Master
    if any("MS" in x for x in degrees):
        return  "Master"
    if any("MA" in x for x in degrees):
        return  "Master"
    if any("LLM" in x for x in degrees):
        return  "Master"
    if any("JD" in x for x in degrees):
        return  "Master"
    if any("M.Eng" in x for x in degrees):
        return  "Master"
    if any("MD" in x for x in degrees):
        return  "Master"
    if any("MSE" in x for x in degrees):
        return  "Master"
    if any("ML" in x for x in degrees):
        return  "Master"

    # Bachelor
    if any("BS" in x for x in degrees):
        return  "Bachelor"
    if any("BA" in x for x in degrees):
        return  "Bachelor"
    if any("LLB" in x for x in degrees):
        return  "Bachelor"
    if any("BE" in x for x in degrees):
        return  "Bachelor"
    if any("BTECH" in x for x in degrees):
        return  "Bachelor"
    if any("BBS" in x for x in degrees):
        return  "Bachelor"
    if any("BBA" in x for x in degrees):
        return  "Bachelor"
    if any("BFA" in x for x in degrees):
        return  "Bachelor"

    # Other
    else:
        return "unknown"

In [38]:
founders_degree = founders_degree.groupby("relationship_object_id").agg({"degree_type": list})

In [39]:
founders_degree = founders_degree.apply(get_degree, axis=1).to_frame("highest_degree_founder")

In [40]:
acquired_df = pd.merge(acquired_df, founders_degree, how="left", left_on="id", right_on="relationship_object_id")

# 6. The final dataset

Create the final dataset containing all the deals with a price higher than 0
To this data we have linked information from the acquirer and acquired companies

In [41]:
df = pd.merge(df, acquirers_df, left_on="acquiring_object_id", right_on="id")

In [42]:
df = pd.merge(df, acquired_df, left_on="acquired_object_id", right_on="id")

In [43]:
df.shape

(21335, 35)

In [44]:
df.drop_duplicates(inplace=True)

In [45]:
df = df[['price_amount', 'price_currency_code', 'acquired_at','name_x', 'category_code_x', 'country_code_x', 'region_x', 'investment_rounds_x', 'invested_companies_x',
         'funding_rounds_x', 'funding_total_usd_x', 'milestones_x', 'relationships_x', 'founded_year_x', 'number_of_offices_x',
         'name_y', 'category_code_y','country_code_y', 'region_y', 'investment_rounds_y', 'invested_companies_y', 'funding_rounds_y', 'funding_total_usd_y', 'milestones_y',
         'relationships_y', 'founded_year_y', 'number_of_investors', 'highest_degree_founder', 'number_of_offices_y'
]]

In [46]:
rename = {
    "name_x": "name_acquirer",
    "category_code_x": "acquirer_industry",
    "country_code_x": "acquirer_country",
    "region_x": "acquirer_region",
    "investment_rounds_x": "acquirer_investment_rounds",
    "invested_companies_x": "acquirer_invested_companies",
    "funding_rounds_x": "acquirer_funding_rounds",
    "funding_total_usd_x": "acquirer_total_funding",
    "milestones_x": "acquirer_milestones",
    "relationships_x": "acquirer_relationships",
    "founded_year_x": "acquirer_founded_year",
    "number_of_offices_x": "acquirer_offices",


    "name_y": "name_acquired",
    "category_code_y": "acquired_industry",
    "country_code_y": "acquired_country",
    "region_y": "acquired_region",
    "investment_rounds_y": "acquired_investment_rounds",
    "invested_companies_y": "acquired_invested_companies",
    "funding_rounds_y": "acquired_funding_rounds",
    "funding_total_usd_y": "acquired_total_funding",
    "milestones_y": "acquired_milestones",
    "relationships_y": "acquired_relationships",
    "founded_year_y": "acquired_founded_year",
    "number_of_investors": "funds_backing_acquired",
    "number_of_offices_y": "acquired_offices",
}

In [47]:
df = df.rename(columns=rename).reset_index(drop=True)

In [48]:
df.to_csv(r"C:\Users\32474\Documents\Ma TEW\masterthesis\data\merged_data.csv")